In [1]:
import pandas as pd
import numpy as np

In [2]:
def create_time_windows(df, window_size, stride):
    # 입력 유효성 검사
    if not isinstance(df, pd.DataFrame):
        raise ValueError("Input must be a pandas DataFrame")
    if not isinstance(window_size, int) or window_size <= 0:
        raise ValueError("window_size must be a positive integer")
    if not isinstance(stride, int) or stride <= 0:
        raise ValueError("stride must be a positive integer")
    
    # 결과를 저장할 딕셔너리 생성
    result_dict = {}
    
    # 각 열에 대해 반복
    for column in df.columns:
        # 시간 윈도우 생성
        windows = {}
        for i in range(0, len(df) - window_size + 1, stride):
            window = df[column].iloc[i:i+window_size].values
            windows[df.index[i+window_size-1]] = window
        
        # 윈도우 데이터로 새 DataFrame 생성
        window_df = pd.DataFrame.from_dict(windows, orient='index')
        window_df.columns = [f"{column}_t-{window_size-i-1}" for i in range(window_size)]
        
        # 결과 딕셔너리에 추가
        result_dict[column] = window_df
    
    return result_dict

In [3]:
df = pd.read_parquet(r"D:\Workspace\DnS\data\AJ네트웍스_20190825_20240825.parquet")

In [4]:
df.shape

(1233, 6)

In [5]:
window_size = 5  # window 크기
stride = 2  # stride 크기
df_list = create_time_windows(df, window_size, stride)

In [6]:
print(df.shape)
print(df.columns)
print(df.head(5))

(1233, 6)
Index(['시가', '고가', '저가', '종가', '거래량', '등락률'], dtype='object')
              시가    고가    저가    종가    거래량       등락률
날짜                                                 
2019-08-26  4615  4615  4480  4540  34971 -2.365591
2019-08-27  4505  4585  4505  4560  20983  0.440529
2019-08-28  4540  4640  4490  4580  20526  0.438596
2019-08-29  4650  4650  4365  4650  22742  1.528384
2019-08-30  4645  4715  4610  4700  20754  1.075269


In [7]:
df_list['종가']

,종가_t-4,종가_t-3,종가_t-2,종가_t-1,종가_t-0
2019-08-30,4540,4560,4580,4650,4700
2019-09-03,4580,4650,4700,4670,4590
2019-09-05,4700,4670,4590,4575,4440
2019-09-09,4590,4575,4440,4455,4375
2019-09-11,4440,4455,4375,4385,4395
...,...,...,...,...,...
2024-08-12,4320,4315,4275,4320,4360
2024-08-14,4275,4320,4360,4325,4350
2024-08-19,4360,4325,4350,4410,4400
2024-08-21,4350,4410,4400,4500,4620


In [8]:
from acf import calculate_acf
calculate_acf(df_list['종가'], window_size=window_size)

,lag_0,lag_1,lag_2,lag_3,lag_4
2019-08-30,1.0,0.403125,-0.153571,-0.403348,-0.346205
2019-09-03,1.0,0.046442,-0.579401,-0.227715,0.260674
2019-09-05,1.0,0.260341,-0.030414,-0.333942,-0.395985
2019-09-09,1.0,0.293466,-0.070114,-0.385350,-0.338002
2019-09-11,1.0,0.205000,-0.330000,-0.285000,-0.090000
...,...,...,...,...,...
2024-08-12,1.0,0.033333,-0.522865,-0.033609,0.023140
2024-08-14,1.0,0.010069,-0.208696,-0.021281,-0.280092
2024-08-19,1.0,0.343426,-0.442629,-0.345219,-0.055578
2024-08-21,1.0,0.269285,-0.116328,-0.269373,-0.383583


In [9]:
from buffett import calculate_buffett_index
buffett = calculate_buffett_index(df['종가'], 'KOR')
create_time_windows(buffett.to_frame(), window_size, stride)['종가']

,종가_t-4,종가_t-3,종가_t-2,종가_t-1,종가_t-0
2019-08-30,2.522222e-07,2.533333e-07,2.544444e-07,2.583333e-07,2.611111e-07
2019-09-03,2.544444e-07,2.583333e-07,2.611111e-07,2.594444e-07,2.550000e-07
2019-09-05,2.611111e-07,2.594444e-07,2.550000e-07,2.541667e-07,2.466667e-07
2019-09-09,2.550000e-07,2.541667e-07,2.466667e-07,2.475000e-07,2.430556e-07
2019-09-11,2.466667e-07,2.475000e-07,2.430556e-07,2.436111e-07,2.441667e-07
...,...,...,...,...,...
2024-08-12,2.400000e-07,2.397222e-07,2.375000e-07,2.400000e-07,2.422222e-07
2024-08-14,2.375000e-07,2.400000e-07,2.422222e-07,2.402778e-07,2.416667e-07
2024-08-19,2.422222e-07,2.402778e-07,2.416667e-07,2.450000e-07,2.444444e-07
2024-08-21,2.416667e-07,2.450000e-07,2.444444e-07,2.500000e-07,2.566667e-07


In [10]:
from deMartini import demartini_index
de = demartini_index(df['종가'])
create_time_windows(de.to_frame(), window_size, stride)['rsi']

,rsi_t-4,rsi_t-3,rsi_t-2,rsi_t-1,rsi_t-0
2019-08-30,NaN,NaN,NaN,NaN,NaN
2019-09-03,NaN,NaN,NaN,NaN,NaN
2019-09-05,NaN,NaN,NaN,NaN,NaN
2019-09-09,NaN,NaN,NaN,NaN,NaN
2019-09-11,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...
2024-08-12,42.934791,42.703634,40.810656,43.827427,46.440592
2024-08-14,40.810656,43.827427,46.440592,44.490296,46.227379
2024-08-19,46.440592,44.490296,46.227379,50.251128,49.585158
2024-08-21,46.227379,50.251128,49.585158,55.881832,62.013127


In [11]:
from div_each_before import div_each_before

deb = div_each_before(df['종가'])
create_time_windows(deb.to_frame(), window_size, stride)['종가']


,종가_t-4,종가_t-3,종가_t-2,종가_t-1,종가_t-0
2019-08-30,1.000000,1.004405,1.004386,1.015284,1.010753
2019-09-03,1.004386,1.015284,1.010753,0.993617,0.982869
2019-09-05,1.010753,0.993617,0.982869,0.996732,0.970492
2019-09-09,0.982869,0.996732,0.970492,1.003378,0.982043
2019-09-11,0.970492,1.003378,0.982043,1.002286,1.002281
...,...,...,...,...,...
2024-08-12,1.032258,0.998843,0.990730,1.010526,1.009259
2024-08-14,0.990730,1.010526,1.009259,0.991972,1.005780
2024-08-19,1.009259,0.991972,1.005780,1.013793,0.997732
2024-08-21,1.005780,1.013793,0.997732,1.022727,1.026667


In [12]:
from fractional_difference import fractional_difference
fracdiff = fractional_difference(df['종가'], 0.3)
create_time_windows(fracdiff.to_frame(), window_size, stride)[0]

,0_t-4,0_t-3,0_t-2,0_t-1,0_t-0
2019-08-30,4540.000000,3198.000000,2735.300000,2527.070000,2370.442250
2019-09-03,2735.300000,2527.070000,2370.442250,2181.169065,1993.661294
2019-09-05,2370.442250,2181.169065,1993.661294,1912.898874,1715.570275
2019-09-09,1993.661294,1912.898874,1715.570275,1713.129101,1591.069188
2019-09-11,1715.570275,1713.129101,1591.069188,1585.326816,1563.758066
...,...,...,...,...,...
2024-08-12,279.225010,261.077750,224.522155,285.203883,319.250241
2024-08-14,224.522155,285.203883,319.250241,272.862348,305.708332
2024-08-19,319.250241,272.862348,305.708332,361.219809,332.767194
2024-08-21,305.708332,361.219809,332.767194,429.816874,517.931556


In [13]:
df['종가']

날짜
2019-08-26    4540
2019-08-27    4560
2019-08-28    4580
2019-08-29    4650
2019-08-30    4700
              ... 
2024-08-19    4400
2024-08-20    4500
2024-08-21    4620
2024-08-22    4610
2024-08-23    4620
Name: 종가, Length: 1233, dtype: int64

In [14]:
from pivot import calculate_pivot_points
pivot_points = calculate_pivot_points(df['고가'], df['저가'], df['종가'])
create_time_windows(pivot_points, window_size, stride)['Pivot']

,Pivot_t-4,Pivot_t-3,Pivot_t-2,Pivot_t-1,Pivot_t-0
2019-08-30,4545.000000,4550.000000,4570.000000,4555.000000,4675.000000
2019-09-03,4570.000000,4555.000000,4675.000000,4671.666667,4610.000000
2019-09-05,4675.000000,4671.666667,4610.000000,4573.333333,4485.000000
2019-09-09,4610.000000,4573.333333,4485.000000,4463.333333,4386.666667
2019-09-11,4485.000000,4463.333333,4386.666667,4381.666667,4391.666667
...,...,...,...,...,...
2024-08-12,4265.000000,4315.000000,4266.666667,4323.333333,4331.666667
2024-08-14,4266.666667,4323.333333,4331.666667,4333.333333,4355.000000
2024-08-19,4331.666667,4333.333333,4355.000000,4396.666667,4405.000000
2024-08-21,4355.000000,4396.666667,4405.000000,4471.666667,4593.333333


In [15]:

from sonar import sonar_indicator
sn = sonar_indicator(df, window_size=14)
create_time_windows(sn.to_frame(), window_size, stride)[0]

,0_t-4,0_t-3,0_t-2,0_t-1,0_t-0
2019-08-30,NaN,0.002747,0.002005,0.002669,0.002416
2019-09-03,0.002005,0.002669,0.002416,0.000920,-0.000085
2019-09-05,0.002416,0.000920,-0.000085,-0.000173,0.010985
2019-09-09,-0.000085,-0.000173,0.010985,0.010359,0.009890
2019-09-11,0.010985,0.010359,0.009890,0.009421,0.008680


In [16]:
from stocastic import stochastic_fast, stochastic_slow
stfa = stochastic_fast(df)

In [17]:
stfa

,fastk,fastd
날짜,,
2019-08-26,44.444444,44.444444
2019-08-27,59.259259,51.851852
2019-08-28,62.500000,55.401235
2019-08-29,100.000000,73.919753
2019-08-30,95.714286,86.071429
...,...,...
2024-08-19,NaN,NaN
2024-08-20,NaN,NaN
2024-08-21,NaN,NaN


In [18]:
create_time_windows(stfa['fastk'].to_frame(), window_size, stride)

{'fastk':             fastk_t-4   fastk_t-3  fastk_t-2  fastk_t-1  fastk_t-0
 2019-08-30  44.444444   59.259259  62.500000      100.0  95.714286
 2019-09-03  62.500000  100.000000  95.714286        NaN        NaN
 2019-09-05  95.714286         NaN        NaN        NaN        NaN
 2019-09-09        NaN         NaN        NaN        NaN        NaN
 2019-09-11        NaN         NaN        NaN        NaN        NaN
 ...               ...         ...        ...        ...        ...
 2024-08-12        NaN         NaN        NaN        NaN        NaN
 2024-08-14        NaN         NaN        NaN        NaN        NaN
 2024-08-19        NaN         NaN        NaN        NaN        NaN
 2024-08-21        NaN         NaN        NaN        NaN        NaN
 2024-08-23        NaN         NaN        NaN        NaN        NaN
 
 [615 rows x 5 columns]}

In [19]:
from time_delay import time_delay_embedding
time_delay_embedding(df['종가'], 103, 7)

,t-0,t-103,t-206,t-309,t-412,t-515,t-618
0,4540,4900,2940,4600,5390,5980,5390
1,4560,4850,2830,4565,5270,5900,5350
2,4580,4905,2860,4610,5340,5850,5480
3,4650,4840,2835,4530,5560,5840,5780
4,4700,4855,2895,4460,5760,5770,5660
...,...,...,...,...,...,...,...
610,5270,6890,5900,4460,3855,4925,4400
611,5350,6900,5850,4475,3740,4930,4500
612,5400,6970,5880,4440,3635,4895,4620
613,5370,7010,5890,4445,3790,4825,4610


In [20]:
from vix import calculate_volatility
calculate_volatility(df_list['종가'], window_size)

AttributeError: 'DataFrame' object has no attribute 'append'

In [ ]:
from williams import williams_r
will = williams_r(df, 5) 
create_time_windows(will.to_frame(), window_size, stride)[0]

,0_t-4,0_t-3,0_t-2,0_t-1,0_t-0
2019-08-30,-55.555556,-40.740741,-37.500000,-0.000000,-4.285714
2019-09-03,-37.500000,-0.000000,-4.285714,-14.084507,-36.619718
2019-09-05,-4.285714,-14.084507,-36.619718,-40.845070,-98.245614
2019-09-09,-36.619718,-40.845070,-98.245614,-91.379310,-85.074627
2019-09-11,-98.245614,-91.379310,-85.074627,-78.571429,-72.549020
...,...,...,...,...,...
2024-08-12,-37.037037,-37.962963,-39.795918,-30.612245,-9.433962
2024-08-14,-39.795918,-30.612245,-9.433962,-34.285714,-24.324324
2024-08-19,-9.433962,-34.285714,-24.324324,-5.882353,-21.052632
2024-08-21,-24.324324,-5.882353,-21.052632,-7.142857,-18.918919
